## Customer attrition(customer churn or 고객 이탈 etc..)

- 본 데이터는 통신사 고객들의 데이터를 결합해 놓은 것입니다
- 내용으로는 서비스 이용 유무(각종 서비스, multiline, 보안, 백업 등), 계약 방식, 지불비용, 총지불비용, 이탈여부 이 있습니다.
- 개수는 총 7043개, colum수는 21개 입니다.
- 데이터 제시 목적은 churn(이탈여부)를 target variable로 하여 어떤 것이 이탈에 영향을 주고 더 나아가 기존 고객 중 이탈 예측 가능성이 있는 고객들을 예측하는 것에 있습니다.

###  EDA

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the "../input/" directory.
import os
import matplotlib.pyplot as plt#visualization
from PIL import  Image
%matplotlib inline
import pandas as pd
import seaborn as sns#visualization
import itertools
import warnings
warnings.filterwarnings("ignore")
import io
import plotly.offline as py#visualization
py.init_notebook_mode(connected=True)#visualization
import plotly.graph_objs as go#visualization
import plotly.tools as tls#visualization
import plotly.figure_factory as ff#visualization


#### 1. loading data

In [ ]:
telcom = pd.read_csv("C:/Users/이만기/Desktop/data/kaggle/WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [ ]:
telcom.head()

#### 1.1 data overview

In [ ]:
print("Rows     : ", telcom.shape[0]) # 행 확인
print("columns     : ", telcom.shape[1]) # 열 확인
print("\nFeautres     : \n", telcom.columns.tolist()) # 열 이름 확인 
print("\nMissing values     : ", telcom.isnull().sum().values.sum()) # missing value 색출
print("\nUnique values     : \n", telcom.nunique()) # 각 열별로 어떻게 구성이 되어 있는지 파악


#### data preprocessing

In [ ]:
# 'total charge' 컬럼에는 값이 입력되지 않은 행들이 있으므로 null값 색출해서 처리해야 한다.
telcom['TotalCharges'] = telcom["TotalCharges"].replace(" ", np.nan)

In [ ]:
# missing value 상태인 것은 날려버리자
telcom = telcom[telcom["TotalCharges"].notnull()]
telcom = telcom.reset_index()[telcom.columns]

In [ ]:
# missing value로 처리되기 전 6531개 였는데 처리하고 난 뒤 전체 데이터 셋의 갯수와 같은 7032개가 되었다.
telcom['TotalCharges']

In [ ]:
# float type으로 데이터 형 변경
telcom["TotalCharges"] = telcom["TotalCharges"].astype(float)

In [ ]:
# replace "No internet service" to No for the following columns
# 아래의 변수들은 범주형 데이터 타입으로 'yes', 'no', 'no internet service'로 구성이 되어있습니다. 
# no 나 no internet serviece나 같은 의미이므로 다 'No'로 처리하겠습니다.
replace_cols = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection','TechSupport','StreamingTV', 'StreamingMovies']

for i in replace_cols:
    telcom[i] = telcom[i].replace({"No internet service": "No"})

In [ ]:
telcom.info()

In [ ]:
# replace values
# 추후 계산의 편의성을 위해 yes or no를 1 or 0으로 변경했습니다.

telcom['SeniorCitizen'] = telcom['SeniorCitizen'].replace({1:"Yes", 0:"No"})

In [ ]:
# Tenure to categorical column
# Tenure은 고객 유지기간을 의미하는데 각 고객마다의 기간이 다르므로 범주화 하였습니다.

def tenure_lab(telcom) :
    
    if telcom["tenure"] <= 12 :
        return "Tenure_0-12"
    elif (telcom["tenure"] > 12) & (telcom["tenure"] <= 24 ):
        return "Tenure_12-24"
    elif (telcom["tenure"] > 24) & (telcom["tenure"] <= 48) :
        return "Tenure_24-48"
    elif (telcom["tenure"] > 48) & (telcom["tenure"] <= 60) :
        return "Tenure_48-60"
    elif telcom["tenure"] > 60 :
        return "Tenure_gt_60"
telcom["tenure_group"] = telcom.apply(lambda telcom:tenure_lab(telcom),
                                      axis = 1)

In [ ]:
telcom["tenure_group"]

In [ ]:
# Seperating churn and non churn customers

churn = telcom[telcom["Churn"] == "Yes"]
not_churn = telcom[telcom["Churn"] == "No"]

In [ ]:
# Seperating catagorical and numerical columns
Id_col = ["customerID"]
target_col = ["Churn"]
cat_cols = telcom.nunique()[telcom.nunique() < 6].keys().tolist()
cat_cols = [x for x in cat_cols if x not in target_col]
num_cols = [x for x in telcom.columns if x not in cat_cols + target_col + Id_col]

In [ ]:
telcom.head()

#### 3. 시각화
#### 3-1. customer attrition in data

In [ ]:
#labels
lab = telcom["Churn"].value_counts().keys().tolist()
#values
val = telcom["Churn"].value_counts().values.tolist()

trace = go.Pie(labels = lab ,
               values = val ,
               marker = dict(colors =  [ 'royalblue' ,'lime'],
                             line = dict(color = "white",
                                         width =  1.3)
                            ),
               rotation = 90,
               hoverinfo = "label+value+text",
               hole = .5
              )
layout = go.Layout(dict(title = "Customer attrition in data",
                        plot_bgcolor  = "rgb(243,243,243)",
                        paper_bgcolor = "rgb(243,243,243)",
                       )
                  )

data = [trace]
fig = go.Figure(data = data,layout = layout)
py.iplot(fig)

- 전체 고객 중 이탈로 반단되는 비율은 26.6%이고 나머지는 73.4%이다.

In [ ]:
# 변수별 그래프를 그립니다.
# columns 에 각 열을 넣었으므로, 여러 코드를 칠 필요 없이 간단하게 작성 가능합니다.

def plot_pie(column) :
    
    trace1 = go.Pie(values  = churn[column].value_counts().values.tolist(),
                    labels  = churn[column].value_counts().keys().tolist(),
                    hoverinfo = "label+percent+name",
                    domain  = dict(x = [0,.48]),
                    name    = "Churn Customers",
                    marker  = dict(line = dict(width = 2,
                                               color = "rgb(243,243,243)")
                                  ),
                    hole    = .6
                   )
    trace2 = go.Pie(values  = not_churn[column].value_counts().values.tolist(),
                    labels  = not_churn[column].value_counts().keys().tolist(),
                    hoverinfo = "label+percent+name",
                    marker  = dict(line = dict(width = 2,
                                               color = "rgb(243,243,243)")
                                  ),
                    domain  = dict(x = [.52,1]),
                    hole    = .6,
                    name    = "Non churn customers" 
                   )


    layout = go.Layout(dict(title = column + " distribution in customer attrition ",
                            plot_bgcolor  = "rgb(243,243,243)",
                            paper_bgcolor = "rgb(243,243,243)",
                            annotations = [dict(text = "churn customers",
                                                font = dict(size = 13),
                                                showarrow = False,
                                                x = .15, y = .5),
                                           dict(text = "Non churn customers",
                                                font = dict(size = 13),
                                                showarrow = False,
                                                x = .88,y = .5
                                               )
                                          ]
                           )
                      )
    data = [trace1,trace2]
    fig  = go.Figure(data = data,layout = layout)
    py.iplot(fig)

# 위는 파이차트였고 여기부터는 히스토그램으로 파악합니다.
#function  for histogram for customer attrition types
def histogram(column) :
    trace1 = go.Histogram(x  = churn[column],
                          histnorm= "percent",
                          name = "Churn Customers",
                          marker = dict(line = dict(width = .5,
                                                    color = "black"
                                                    )
                                        ),
                         opacity = .9 
                         ) 
    
    trace2 = go.Histogram(x  = not_churn[column],
                          histnorm = "percent",
                          name = "Non churn customers",
                          marker = dict(line = dict(width = .5,
                                              color = "black"
                                             )
                                 ),
                          opacity = .9
                         )
    
    data = [trace1,trace2]
    layout = go.Layout(dict(title =column + " distribution in customer attrition ",
                            plot_bgcolor  = "rgb(243,243,243)",
                            paper_bgcolor = "rgb(243,243,243)",
                            xaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                             title = column,
                                             zerolinewidth=1,
                                             ticklen=5,
                                             gridwidth=2
                                            ),
                            yaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                             title = "percent",
                                             zerolinewidth=1,
                                             ticklen=5,
                                             gridwidth=2
                                            ),
                           )
                      )
    fig  = go.Figure(data=data,layout=layout)
    
    py.iplot(fig)
    
    
# 산점도로 파악해 보겠습니다.

def scatter_matrix(df)  :
    
    df  = df.sort_values(by = "Churn" ,ascending = True)
    classes = df["Churn"].unique().tolist()
    classes
    
    class_code  = {classes[k] : k for k in range(2)}
    class_code

    color_vals = [class_code[cl] for cl in df["Churn"]]
    color_vals

    pl_colorscale = "Portland"

    pl_colorscale

    text = [df.loc[k,"Churn"] for k in range(len(df))]
    text

    trace = go.Splom(dimensions = [dict(label  = "tenure",
                                       values = df["tenure"]),
                                  dict(label  = 'MonthlyCharges',
                                       values = df['MonthlyCharges']),
                                  dict(label  = 'TotalCharges',
                                       values = df['TotalCharges'])],
                     text = text,
                     marker = dict(color = color_vals,
                                   colorscale = pl_colorscale,
                                   size = 3,
                                   showscale = False,
                                   line = dict(width = .1,
                                               color='rgb(230,230,230)'
                                              )
                                  )
                    )
    axis = dict(showline  = True,
                zeroline  = False,
                gridcolor = "#fff",
                ticklen   = 4
               )
    
    layout = go.Layout(dict(title  = 
                            "Scatter plot matrix for Numerical columns for customer attrition",
                            autosize = False,
                            height = 800,
                            width  = 800,
                            dragmode = "select",
                            hovermode = "closest",
                            plot_bgcolor  = 'rgba(240,240,240, 0.95)',
                            xaxis1 = dict(axis),
                            yaxis1 = dict(axis),
                            xaxis2 = dict(axis),
                            yaxis2 = dict(axis),
                            xaxis3 = dict(axis),
                            yaxis3 = dict(axis),
                           )
                      )
    data   = [trace]
    fig = go.Figure(data = data,layout = layout )
    py.iplot(fig)

#for all categorical columns plot pie
for i in cat_cols :
    plot_pie(i)

#for all categorical columns plot histogram    
for i in num_cols :
    histogram(i)

#scatter plot matrix
scatter_matrix(telcom)

#### 3-2. 위에서 작업한 고객기간 별 고객 분리를 바탕으로 시각화를 진행해보겠습니다.

In [ ]:
# 이탈과 비이탈에 해당하는 변수를 새로 생성하고 진행합니다.


tg_ch  =  churn["tenure_group"].value_counts().reset_index()
tg_ch.columns  = ["tenure_group","count"]
tg_nch =  not_churn["tenure_group"].value_counts().reset_index()
tg_nch.columns = ["tenure_group","count"]

#bar - churn
trace1 = go.Bar(x = tg_ch["tenure_group"]  , y = tg_ch["count"],
                name = "Churn Customers",
                marker = dict(line = dict(width = .5,color = "black")),
                opacity = .9)

#bar - not churn
trace2 = go.Bar(x = tg_nch["tenure_group"] , y = tg_nch["count"],
                name = "Non Churn Customers",
                marker = dict(line = dict(width = .5,color = "black")),
                opacity = .9)

layout = go.Layout(dict(title = "Customer attrition in tenure groups",
                        plot_bgcolor  = "rgb(243,243,243)",
                        paper_bgcolor = "rgb(243,243,243)",
                        xaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                     title = "tenure group",
                                     zerolinewidth=1,ticklen=5,gridwidth=2),
                        yaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                     title = "count",
                                     zerolinewidth=1,ticklen=5,gridwidth=2),
                       )
                  )
data = [trace1,trace2]
fig  = go.Figure(data=data,layout=layout)
py.iplot(fig)

In [ ]:
# 정확한 분석을 해야 알겠지만, 대략적으로 고객기간이 길 수록 이탈 고객의 비율이 줄어든다는 것입니다.

print(tg_ch)
print(tg_nch)

#### 3-4. 월별 청구액과 총 청구액을 고객기간과 이탈 유무 그룹에 따라 시각화하기

In [ ]:
telcom[['MonthlyCharges', 'TotalCharges','tenure',"tenure_group"]]

#분포를 보기 위해 산점도로 시각화해보겠습니다.
# 고객 기간에 따른 총 지불비용
def plot_tenure_scatter(tenure_group,color) :
    tracer = go.Scatter(x = telcom[telcom["tenure_group"] == tenure_group]["MonthlyCharges"],
                        y = telcom[telcom["tenure_group"] == tenure_group]["TotalCharges"], # x축과 y축을 지정하고
                        mode = "markers",marker = dict(line = dict(color = "black",
                                                                   width = .2),
                                                       size = 4 , color = color,
                                                       symbol = "diamond-dot",
                                                      ),
                        name = tenure_group,
                        opacity = .9
                       )
    return tracer


# 월별 지불비용에 따른 총 지불비용 
def plot_churncharges_scatter(churn,color) :
    tracer = go.Scatter(x = telcom[telcom["Churn"] == churn]["MonthlyCharges"],
                        y = telcom[telcom["Churn"] == churn]["TotalCharges"],
                        mode = "markers",marker = dict(line = dict(color = "black",
                                                                   width = .2),
                                                       size = 4 , color = color,
                                                       symbol = "diamond-dot",
                                                      ),
                        name = "Churn - " + churn,
                        opacity = .9
                       )
    return tracer

# 색을 지정해 줄 수 있습니다. 귀찮은 과정이긴 하지만, 시각적 효과를 높이기 위해 필요한 부분입니다.
trace1 = plot_tenure_scatter("Tenure_0-12","#FF3300")
trace2 = plot_tenure_scatter("Tenure_12-24","#6666FF")
trace3 = plot_tenure_scatter("Tenure_24-48","#99FF00")
trace4 = plot_tenure_scatter("Tenure_48-60","#996600")
trace5 = plot_tenure_scatter("Tenure_gt_60","grey")
trace6 = plot_churncharges_scatter("Yes","red")
trace7 = plot_churncharges_scatter("No","blue")

data1   = [trace1,trace2,trace3,trace4,trace5] 
data2   = [trace7,trace6]

#layout
def layout_title(title) :
    layout = go.Layout(dict(title = title,
                            plot_bgcolor  = "rgb(243,243,243)",
                            paper_bgcolor = "rgb(243,243,243)",
                            xaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                         title = "Monthly charges",
                                         zerolinewidth=1,ticklen=5,gridwidth=2),
                            yaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                         title = "Total Charges",
                                         zerolinewidth=1,ticklen=5,gridwidth=2),
                            height = 600
                           )
                      )
    return layout

layout1  = layout_title("Monthly Charges & Total Charges by Tenure group")
layout2  = layout_title("Monthly Charges & Total Charges by Churn group")
fig1 = go.Figure(data = data1,layout = layout1)
fig2 = go.Figure(data = data2,layout = layout2)
py.iplot(fig1)
py.iplot(fig2)

- 두 그래프를 통해 알 수 있는 것은,
- 고객기간이 긴 고객의 경우 지불비용 또한 높게 형성되어있는 것을 확인할 수 있습니다.
- 아래의 그래프를 통해 이탈하지 않은 고객의 경우 총 지불비용과 월별 지불비용이 이탈한 고객들에 비해 상대적으로 높은 것으로 나타났습니다
- 이러한 것들은 상식적으로 생각할 수 있는 부분이지만, 실제 시각화를 통해 눈으로 확인할 수 있다는 점에서 의미가 있습니다.

## Data preprocessing

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [ ]:
# customer id col
Id_col = ['customerID']

# target columns
target_col = ['Churn']

# categorical columns
# tolist : 배열을 목록으로 변환

cat_cols = telcom.nunique()[telcom.nunique() < 6].keys().tolist() # 고유한 값이 6개 미만인 변수들을 찾아 가져와라
cat_cols = [x for x in cat_cols if x not in target_col]

In [ ]:
cat_cols

In [ ]:
# numerical columns
# 수치형 만으로 구성된 컬럼들을 찾기 위한 과정

num_cols = [x for x in telcom.columns if x not in cat_cols + target_col + Id_col]

# Binary columnms with 2 values
# 2개의 범주 vs 3개 이상의 범주로 구성된 변수를 나눕니다.
bin_cols = telcom.nunique()[telcom.nunique() == 2].keys().tolist()

# Columns more that 2 values
multi_cols = [i for i in cat_cols if i not in bin_cols]

# label encoding binary columns
le = LabelEncoder() # 보다 편안한 방식으로 범주에 0, 1, 2 등과 같은 binary로 변환해줌
for i in bin_cols :
    telcom[i] = le.fit_transform(telcom[i])
    
# Duplicating columns for multi value columns
telcom = pd.get_dummies(data = telcom, columns = multi_cols)

# Scaling Numeriocal columns
# 정규화를 진행하기 위해 standardscaler를 사용
std = StandardScaler()
scaled = std.fit_transform(telcom[num_cols]) 
scaled = pd.DataFrame(scaled, columns = num_cols)

# dropping original values merging scaled values for numerical columns
df_telcom_og = telcom.copy()
telcom = telcom.drop(columns = num_cols, axis = 1) # 기존 정규화가 진행되지 않은 변수들을 drop하고 scaled를 집어 넣음
telcom = telcom.merge(scaled, left_index = True, right_index = True, how = 'left')

In [ ]:
telcom['tenure']

## 변수별 요약

In [ ]:
summary = (df_telcom_og[[i for i in df_telcom_og.columns if i not in Id_col]].describe().transpose().reset_index())

In [ ]:
summary

In [ ]:
summary = (df_telcom_og[[i for i in df_telcom_og.columns if i not in Id_col]].
           describe().transpose().reset_index())

summary = summary.rename(columns = {"index" : "feature"})
summary = np.around(summary,3)

val_lst = [summary['feature'], summary['count'],
           summary['mean'],summary['std'],
           summary['min'], summary['25%'],
           summary['50%'], summary['75%'], summary['max']]

trace  = go.Table(header = dict(values = summary.columns.tolist(),
                                line = dict(color = ['#506784']),
                                fill = dict(color = ['#119DFF']),
                               ),
                  cells  = dict(values = val_lst,
                                line = dict(color = ['#506784']),
                                fill = dict(color = ["lightgrey",'#F5F8FF'])
                               ),
                  columnwidth = [200,60,100,100,60,60,80,80,80])
layout = go.Layout(dict(title = "Variable Summary"))
figure = go.Figure(data=[trace],layout=layout)
py.iplot(figure)

## Correlation Matrix

In [ ]:
# correlation
correlation = telcom.corr()
print(correlation)
# tick labels 
matrix_cols = correlation.columns.tolist()
print(matrix_cols)
# convert to array
corr_array = np.array(correlation)
print(corr_array)

In [ ]:
# 너무 번잡스러우니 시각화 해보겠습니다.

#Plotting
trace = go.Heatmap(z = corr_array,
                   x = matrix_cols,
                   y = matrix_cols,
                   colorscale = "Viridis",
                   colorbar   = dict(title = "Pearson Correlation coefficient",
                                     titleside = "right"
                                    ) ,
                  )

layout = go.Layout(dict(title = "Correlation Matrix for variables",
                        autosize = False,
                        height  = 720,
                        width   = 800,
                        margin  = dict(r = 0 ,l = 210,
                                       t = 25,b = 210,
                                      ),
                        yaxis   = dict(tickfont = dict(size = 9)),
                        xaxis   = dict(tickfont = dict(size = 9))
                       )
                  )

data = [trace]
fig = go.Figure(data=data,layout=layout)
py.iplot(fig)

#### 알수있는 것
- 월별 지불비용과 사용할 수록 과금이 되는 서비스 들간의 상관관계가 다른 항목들에 비해 높다.
- 스트리밍 관련 서비스와 백업 등의 보안 관련 서비스 사이의 상관관계가 비교적 높다
- 고객 기간과 총 지불 금액 사이의 관계가 높은 것으로 보인다.

## 고객 이탈 여부에 따라 변수의 분포 파악해보기

In [ ]:
# seperating binary columns

bi_cs = telcom.nunique()[telcom.nunique() == 2].keys() # unique 한 변수 중 2개를 범주로 갖는 변수 선택
dat_rad = telcom[bi_cs]

In [ ]:
#plotting radar chart for churn and non churn customers(binary variables)
def plot_radar(df,aggregate,title) :
    data_frame = df[df["Churn"] == aggregate] 
    data_frame_x = data_frame[bi_cs].sum().reset_index()
    data_frame_x.columns  = ["feature","yes"]
    data_frame_x["no"]    = data_frame.shape[0]  - data_frame_x["yes"] # shape[0]은 (n,m) 중 n을 가져오라는 의미
    data_frame_x  = data_frame_x[data_frame_x["feature"] != "Churn"]
    
    #count of 1's(yes)
    trace1 = go.Scatterpolar(r = data_frame_x["yes"].values.tolist(),
                             theta = data_frame_x["feature"].tolist(),
                             fill  = "toself",name = "count of 1's",
                             mode = "markers+lines",
                             marker = dict(size = 5)
                            )
    #count of 0's(No)
    trace2 = go.Scatterpolar(r = data_frame_x["no"].values.tolist(),
                             theta = data_frame_x["feature"].tolist(),
                             fill  = "toself",name = "count of 0's",
                             mode = "markers+lines",
                             marker = dict(size = 5)
                            ) 
    layout = go.Layout(dict(polar = dict(radialaxis = dict(visible = True,
                                                           side = "counterclockwise",
                                                           showline = True,
                                                           linewidth = 2,
                                                           tickwidth = 2,
                                                           gridcolor = "white",
                                                           gridwidth = 2),
                                         angularaxis = dict(tickfont = dict(size = 10),
                                                            layer = "below traces"
                                                           ),
                                         bgcolor  = "rgb(243,243,243)",
                                        ),
                            paper_bgcolor = "rgb(243,243,243)",
                            title = title,height = 700))
    
    data = [trace2,trace1]
    fig = go.Figure(data=data,layout=layout)
    py.iplot(fig)

#plot
plot_radar(dat_rad,1,"Churn -  Customers")
plot_radar(dat_rad,0,"Non Churn - Customers")

### 이탈 vs 비이탈
- 이탈한 고객과 비이탈한 고객의 특성을 알아보면
- 이탈 고객군의 경우 비용지 청구서 서비스를 상대적으로 많이 신청하고 있었다.
- 인터넷 서비스, 월별 계약, 폰서비스의 경우 많이 사용하고 있었다.

- 비이탈 고객군의 경우, 비용지 청구서의 신청 비율이 비슷했고, 폰서비스의 경우 사용하는 사람이 사용하지 않는 사람에 비해 월등히 많았다. 이는 이탈 고객군에서도 같은 현상으로 나타난다.
